In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import zlib

sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))

from proto import video_pb2

# Setting path

In [2]:
store_file_path = '../proto/VIDEO_STORE_FILE_ZLIB'

input_video_path = '../data/test_video.mp4'
output_video_path = '../data/test_video_d.mp4'

# Serialize

In [3]:
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    sys.exit()

In [4]:
video = video_pb2.Video()

In [5]:
def add_video_image(image, img):
    image.height = img.shape[0]
    image.width = img.shape[1]
    image.channel = img.shape[2]
    image.data = np.ndarray.tobytes(img) 

In [6]:
while True:
    ret, img = cap.read()
    if not ret:
        break

    add_video_image(video.image.add(), img)

cap.release()

In [7]:
try:
    os.remove(file_name)
except:
    pass

In [8]:
with open(store_file_path, "wb") as f:
  f.write(zlib.compress(video.SerializeToString()))

# Deserialize

In [9]:
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
out = cv2.VideoWriter(output_video_path, fourcc, 30, (1280, 720))
if not out.isOpened():
    sys.exit()

In [10]:
video_d = video_pb2.Video()

In [11]:
with open(store_file_path, "rb") as pb:
    video_d.ParseFromString(zlib.decompress(pb.read()))

In [12]:
for image in video_d.image:
    img_np = np.frombuffer(image.data, np.uint8)
    img_np = img_np.reshape(image.height, image.width, image.channel)

    out.write(img_np)

    cv2.imshow('deserialized_video', img_np)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cv2.destroyAllWindows()